<a href="https://colab.research.google.com/github/jfraska/smart-cowshed/blob/master/smart_cowshed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import system libs
import os
import time
import shutil
import itertools

# import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt

# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage import io
from PIL import Image
import random
from shutil import copyfile

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

print ('modules loaded')

modules loaded


In [ ]:
# Get dataset
os.environ['KAGGLE_USERNAME'] = "alifianzulfaani"
os.environ['KAGGLE_KEY'] = "4b8aa99d8f0cfab2b85c684ad6864489"
!kaggle datasets download -d alifianzulfaani/new-cowshed-dataset

# Extract dataset
from zipfile import ZipFile
file_name = "/content/new-cowshed-dataset.zip"
zp = ZipFile(file_name, 'r')
zp.extractall(path='/content/')
print('done')